In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
%run "6. Modelo robusto.ipynb"

In [ ]:
def Evaluar_Modelo_Robusto(Resultados_Modelo, Mostrar_Detalles_Tecnicos=True):

    """
    Interpreta y evalúa los resultados de un modelo de regresión lineal robusto,
    proporcionando análisis comprensible de validez metodológica, calidad de ajuste,
    significancia de predictores y recomendaciones prácticas basadas en criterios
    estadísticos rigurosos.
    
    Esta función traduce resultados técnicos complejos en interpretaciones claras
    y actionables, evaluando automáticamente la validez del modelo mediante
    múltiples criterios estadísticos y proporcionando recomendaciones específicas
    para mejorar el análisis o interpretar correctamente los hallazgos.
    
    La evaluación incluye análisis de:
    - Calidad de ajuste y poder explicativo
    - Validez de supuestos estadísticos fundamentales  
    - Significancia e interpretación de coeficientes
    - Presencia de multicolinealidad residual
    - Detección de observaciones influyentes
    - Adecuación del tamaño muestral
    - Estabilidad numérica del modelo
    
    Parámetros:
    -----------
    Resultados_Modelo : dict
        Diccionario con resultados completos del modelo robusto generado por
        la función Modelo_Lineal_Robusto().
        Debe contener todas las claves estándar de salida.
    
    Mostrar_Detalles_Tecnicos : bool, opcional (default=True)
        Si True, incluye sección con diagnósticos técnicos detallados
        (VIF, tests estadísticos, métricas de ajuste avanzadas).
        Si False, se enfoca solo en interpretación práctica.
    
    Retorna:
    --------
    dict
        Diccionario con evaluación estructurada del modelo:
        
        - 'Validez_General': str - "VÁLIDO", "VÁLIDO_CON_RESERVAS", "NO_VÁLIDO"
        - 'Calidad_Ajuste': str - "EXCELENTE", "BUENA", "MODERADA", "POBRE"
        - 'Cumplimiento_Supuestos': dict - Estado de cada supuesto estadístico
        - 'Interpretaciones_Coeficientes': dict - Interpretación de cada predictor
        - 'Recomendaciones_Principales': list - Acciones sugeridas
        - 'Limitaciones_Identificadas': list - Advertencias importantes
        - 'Puntuacion_Confianza': float - Índice de confianza (0-100)
    
    Ejemplo de uso:
    ---------------
    >>> # Después de ajustar modelo robusto
    >>> Resultados = Modelo_Lineal_Robusto(X, y, 'Variable')
    >>> 
    >>> # Interpretar resultados de forma comprensible
    >>> Evaluacion = Interpretar_Y_Evaluar_Modelo_Robusto(
    ...     Resultados, 
    ...     Mostrar_Detalles_Tecnicos=True
    ... )
    >>> 
    >>> print(f"Validez del modelo: {Evaluacion['Validez_General']}")
    >>> print(f"Confianza: {Evaluacion['Puntuacion_Confianza']:.0f}%")
    
    Notas metodológicas:
    --------------------
    - La función aplica criterios estadísticos estándar para evaluar cada
      aspecto del modelo de forma sistemática y objetiva.
    
    - Las interpretaciones consideran tanto significancia estadística como
      relevancia práctica de los efectos encontrados.
    
    - Las recomendaciones priorizan mejoras metodológicas y advertencias
      sobre limitaciones que podrían afectar conclusiones.
    
    - La puntuación de confianza integra múltiples indicadores de calidad
      en un índice comprensible (0-100%).
    
    """
    
    if Resultados_Modelo is None:
        print("\n❌ ERROR: El modelo no produjo resultados válidos")
        print("No es posible realizar interpretación")
        return None
    
    print(f"\n{'🔍 INTERPRETACIÓN INTELIGENTE DEL MODELO ROBUSTO':^80}")
    print("="*80)
    print(f"Variable analizada: {Resultados_Modelo['Variable_Dependiente']}")
    print(f"Observaciones: {Resultados_Modelo['N_Observaciones']:,}")
    print(f"Iteraciones realizadas: {Resultados_Modelo['Iteraciones_Totales']}")
    print("="*80)
    
    # SECCIÓN 1: EVALUACIÓN GENERAL DE VALIDEZ DEL MODELO.
    print(f"\n📊 1. EVALUACIÓN GENERAL DE VALIDEZ")
    print("-"*50)
    
    # Criterios de validez general.
    R2_Ajustado = Resultados_Modelo['R_Cuadrado_Ajustado']
    N_Variables = len(Resultados_Modelo['Variables_Significativas'])
    N_Obs = Resultados_Modelo['N_Observaciones']
    P_Valor_Modelo = Resultados_Modelo['P_Valor_Modelo']
    
    # Evaluar criterios básicos.
    Criterios_Validez = {
        'Variables_Significativas': N_Variables > 0,
        'Modelo_Globalmente_Significativo': P_Valor_Modelo < 0.05,
        'Tamaño_Muestra_Adecuado': N_Obs >= (N_Variables * 10 + 50),
        'R2_Minimo_Aceptable': R2_Ajustado >= 0.10
    }
    
    Criterios_Cumplidos = sum(Criterios_Validez.values())
    Total_Criterios = len(Criterios_Validez)
    
    # Determinar validez general.
    if Criterios_Cumplidos == Total_Criterios:
        Validez_General = "✅ MODELO VÁLIDO"
        Color_Validez = "🟢"
    elif Criterios_Cumplidos >= Total_Criterios * 0.75:
        Validez_General = "⚠️ MODELO VÁLIDO CON RESERVAS"
        Color_Validez = "🟡"
    else:
        Validez_General = "❌ MODELO NO VÁLIDO"
        Color_Validez = "🔴"
    
    print(f"{Color_Validez} {Validez_General}")
    print(f"Criterios cumplidos: {Criterios_Cumplidos}/{Total_Criterios}")
    
    # Detallar cada criterio.
    for Criterio, Cumple in Criterios_Validez.items():
        Icono = "✅" if Cumple else "❌"
        if Criterio == 'Variables_Significativas':
            Detalle = f"({N_Variables} variables significativas)"
        elif Criterio == 'Modelo_Globalmente_Significativo':
            Detalle = f"(p = {P_Valor_Modelo:.6f})"
        elif Criterio == 'Tamaño_Muestra_Adecuado':
            Minimo_Recomendado = N_Variables * 10 + 50
            Detalle = f"({N_Obs:,} obs, mínimo recomendado: {Minimo_Recomendado:,})"
        elif Criterio == 'R2_Minimo_Aceptable':
            Detalle = f"(R² = {R2_Ajustado:.3f})"
        
        print(f"  {Icono} {Criterio.replace('_', ' ')}: {Detalle}")
    
    # SECCIÓN 2: CALIDAD DEL AJUSTE.
    print(f"\n📈 2. CALIDAD DEL AJUSTE Y PODER EXPLICATIVO")
    print("-"*50)
    
    # Categorizar calidad según R² ajustado.
    if R2_Ajustado >= 0.70:
        Calidad_Ajuste = "🟢 EXCELENTE"
        Interpretacion_R2 = "El modelo explica una proporción muy alta de la variabilidad"
    elif R2_Ajustado >= 0.50:
        Calidad_Ajuste = "🟢 BUENA"
        Interpretacion_R2 = "El modelo tiene buen poder explicativo"
    elif R2_Ajustado >= 0.30:
        Calidad_Ajuste = "🟡 MODERADA"
        Interpretacion_R2 = "El modelo explica una proporción moderada de la variabilidad"
    elif R2_Ajustado >= 0.10:
        Calidad_Ajuste = "🟠 BAJA"
        Interpretacion_R2 = "El modelo tiene poder explicativo limitado pero detectable"
    else:
        Calidad_Ajuste = "🔴 MUY BAJA"
        Interpretacion_R2 = "El modelo explica muy poca variabilidad"
    
    print(f"Calidad del ajuste: {Calidad_Ajuste}")
    print(f"R² ajustado: {R2_Ajustado:.3f} ({R2_Ajustado*100:.1f}%)")
    print(f"Interpretación: {Interpretacion_R2}")
    
    # Comparar métricas de información.
    AIC = Resultados_Modelo['AIC']
    BIC = Resultados_Modelo['BIC']
    print(f"\nMétricas de selección de modelos:")
    print(f"  AIC: {AIC:.2f} (menor es mejor)")
    print(f"  BIC: {BIC:.2f} (menor es mejor, más conservador)")
    
    if BIC < AIC:
        print(f"  ℹ️  BIC prefiere modelos más simples que AIC")
    
    # SECCIÓN 3: ANÁLISIS DE SUPUESTOS ESTADÍSTICOS.
    print(f"\n🔬 3. VALIDACIÓN DE SUPUESTOS ESTADÍSTICOS")
    print("-"*50)
    
    # Evaluar cada supuesto.
    Supuestos = {}
    
    # Normalidad de residuos (Jarque-Bera).
    JB_Stat, JB_Pval = Resultados_Modelo['Test_Normalidad_JB']
    Supuestos['Normalidad'] = {
        'Cumple': JB_Pval >= 0.05,
        'P_Valor': JB_Pval,
        'Interpretacion': "Residuos siguen distribución normal" if JB_Pval >= 0.05 else "Residuos NO siguen distribución normal"
    }
    
    # Homocedasticidad (Breusch-Pagan).
    BP_Stat, BP_Pval = Resultados_Modelo['Test_Heterocedasticidad_BP']
    Supuestos['Homocedasticidad'] = {
        'Cumple': BP_Pval >= 0.05,
        'P_Valor': BP_Pval,
        'Interpretacion': "Varianza constante de residuos" if BP_Pval >= 0.05 else "Presencia de heterocedasticidad"
    }
    
    # Independencia (Durbin-Watson).
    DW = Resultados_Modelo['Durbin_Watson']
    DW_OK = 1.5 <= DW <= 2.5  # Rango aceptable.
    Supuestos['Independencia'] = {
        'Cumple': DW_OK,
        'Estadistico': DW,
        'Interpretacion': "Sin autocorrelación" if DW_OK else "Posible autocorrelación en residuos"
    }
    
    # Multicolinealidad (VIF).
    VIF_Valores = list(Resultados_Modelo['VIF_Final'].values()) if Resultados_Modelo['VIF_Final'] else []
    Max_VIF = max(VIF_Valores) if VIF_Valores else 1.0
    VIF_OK = Max_VIF <= 5.0
    Supuestos['Multicolinealidad'] = {
        'Cumple': VIF_OK,
        'Max_VIF': Max_VIF,
        'Interpretacion': "Sin multicolinealidad severa" if VIF_OK else "Presencia de multicolinealidad"
    }
    
    # Reporte de supuestos.
    Supuestos_Cumplidos = sum(1 for s in Supuestos.values() if s['Cumple'])
    Total_Supuestos = len(Supuestos)
    
    print(f"Supuestos cumplidos: {Supuestos_Cumplidos}/{Total_Supuestos}")
    
    for Nombre, Datos in Supuestos.items():
        Icono = "✅" if Datos['Cumple'] else "⚠️"
        
        if Nombre == 'Normalidad':
            Detalle = f"(Jarque-Bera p = {Datos['P_Valor']:.6f})"
        elif Nombre == 'Homocedasticidad':
            Detalle = f"(Breusch-Pagan p = {Datos['P_Valor']:.6f})"
        elif Nombre == 'Independencia':
            Detalle = f"(Durbin-Watson = {Datos['Estadistico']:.3f})"
        elif Nombre == 'Multicolinealidad':
            Detalle = f"(VIF máximo = {Datos['Max_VIF']:.2f})"
        
        print(f"  {Icono} {Nombre}: {Datos['Interpretacion']} {Detalle}")
    
    # SECCIÓN 4: INTERPRETACIÓN DE VARIABLES SIGNIFICATIVAS.
    print(f"\n🎯 4. INTERPRETACIÓN DE VARIABLES SIGNIFICATIVAS")
    print("-"*50)
    
    Variables_Sig = Resultados_Modelo['Variables_Significativas']
    Coeficientes = Resultados_Modelo['Coeficientes']
    P_Valores = Resultados_Modelo['P_Valores']
    Intervalos = Resultados_Modelo['Intervalos_Confianza_95']
    
    if len(Variables_Sig) == 0:
        print("❌ No se encontraron variables estadísticamente significativas")
        print("   El modelo no tiene predictores válidos")
    else:
        print(f"Variables significativas encontradas: {len(Variables_Sig)}")
        print(f"Variables eliminadas del modelo original: {len(Resultados_Modelo['Variables_Eliminadas_Total'])}")
        
        # Interpretar cada coeficiente.
        Interpretaciones_Variables = {}
        
        for Variable in Variables_Sig:
            if Variable in Coeficientes:
                Beta = Coeficientes[Variable]
                P_Val = P_Valores[Variable]
                
                # Obtener intervalo de confianza.
                if Variable in Intervalos:
                    IC_Dict = Intervalos[Variable]
                    IC_Inf = list(IC_Dict.values())[0]  # Límite inferior
                    IC_Sup = list(IC_Dict.values())[1]  # Límite superior
                else:
                    IC_Inf = IC_Sup = None
                
                # Determinar magnitud del efecto.
                Magnitud_Abs = abs(Beta)
                if Magnitud_Abs >= 1.0:
                    Magnitud_Desc = "GRANDE"
                elif Magnitud_Abs >= 0.5:
                    Magnitud_Desc = "MODERADO"
                elif Magnitud_Abs >= 0.2:
                    Magnitud_Desc = "PEQUEÑO"
                else:
                    Magnitud_Desc = "MUY PEQUEÑO"
                
                # Determinar dirección.
                Direccion = "POSITIVO" if Beta > 0 else "NEGATIVO"
                
                # Construir interpretación.
                if IC_Inf is not None and IC_Sup is not None:
                    Interpretacion = f"Efecto {Direccion} {Magnitud_Desc} (β = {Beta:.4f}, p = {P_Val:.6f})"
                    Interpretacion += f"\n      IC 95%: [{IC_Inf:.4f}, {IC_Sup:.4f}]"
                    
                    # Verificar si el IC incluye cero (no debería para variables significativas).
                    if IC_Inf * IC_Sup < 0:
                        Interpretacion += " ⚠️ IC incluye cero"
                else:
                    Interpretacion = f"Efecto {Direccion} {Magnitud_Desc} (β = {Beta:.4f}, p = {P_Val:.6f})"
                
                Interpretaciones_Variables[Variable] = {
                    'Beta': Beta,
                    'P_Valor': P_Val,
                    'Magnitud': Magnitud_Desc,
                    'Direccion': Direccion,
                    'Interpretacion_Completa': Interpretacion
                }
                
                print(f"  📌 {Variable}:")
                print(f"      {Interpretacion}")
    
    # SECCIÓN 5: OUTLIERS Y OBSERVACIONES INFLUYENTES.
    print(f"\n🔍 5. ANÁLISIS DE OBSERVACIONES INFLUYENTES")
    print("-"*50)
    
    N_Outliers = Resultados_Modelo['N_Observaciones_Outliers']
    Porcentaje_Outliers = (N_Outliers / N_Obs) * 100
    
    if N_Outliers == 0:
        print("✅ No se detectaron observaciones influyentes")
        print("   Los datos parecen estar libres de valores extremos problemáticos")
    else:
        print(f"⚠️ Detectadas {N_Outliers} observaciones influyentes ({Porcentaje_Outliers:.1f}%)")
        
        if Porcentaje_Outliers <= 5:
            print("   Porcentaje aceptable de outliers (≤5%)")
        elif Porcentaje_Outliers <= 10:
            print("   Porcentaje moderado de outliers (5-10%)")
        else:
            print("   Porcentaje alto de outliers (>10%) - requiere investigación")
        
        print("   Recomendación: Examinar estas observaciones para errores o casos especiales")
    
    # SECCIÓN 6: DETALLES TÉCNICOS (si se solicitan).
    if Mostrar_Detalles_Tecnicos:
        print(f"\n🔧 6. DIAGNÓSTICOS TÉCNICOS DETALLADOS")
        print("-"*50)
        
        print(f"Número de condición: {Resultados_Modelo['Numero_Condicion']:.1f}")
        if Resultados_Modelo['Numero_Condicion'] > 30:
            print("  ⚠️ Posible inestabilidad numérica (>30)")
        
        print(f"Estadístico F global: {Resultados_Modelo['F_Estadistico']:.3f}")
        
        if Resultados_Modelo['VIF_Final']:
            print(f"\nFactores de Inflación de Varianza (VIF):")
            for Variable, VIF in Resultados_Modelo['VIF_Final'].items():
                Icono = "⚠️" if VIF > 5 else "✅"
                print(f"  {Icono} {Variable}: {VIF:.2f}")
        
        print(f"\nProceso de eliminación:")
        print(f"  Variables eliminadas por VIF: {len(Resultados_Modelo['Variables_Eliminadas_VIF'])}")
        Total_Eliminadas = len(Resultados_Modelo['Variables_Eliminadas_Total'])
        Eliminadas_Por_Seleccion = Total_Eliminadas - len(Resultados_Modelo['Variables_Eliminadas_VIF'])
        print(f"  Variables eliminadas por selección: {Eliminadas_Por_Seleccion}")
        print(f"  Criterio utilizado: {Resultados_Modelo['Criterio_Utilizado']}")
    
    # SECCIÓN 7: RECOMENDACIONES Y CONCLUSIONES.
    print(f"\n💡 7. RECOMENDACIONES Y CONCLUSIONES")
    print("-"*50)
    
    Recomendaciones = []
    Limitaciones = []
    
    # Recomendaciones basadas en validez.
    if Validez_General.startswith("✅"):
        Recomendaciones.append("El modelo es estadísticamente válido y puede usarse para inferencia")
    elif Validez_General.startswith("⚠️"):
        Recomendaciones.append("El modelo tiene limitaciones pero puede ser útil con precauciones")
        Limitaciones.append("Interpretar resultados con cautela debido a limitaciones metodológicas")
    else:
        Recomendaciones.append("El modelo requiere mejoras significativas antes de usarse")
        Limitaciones.append("Modelo no válido para inferencia estadística")
    
    # Recomendaciones basadas en supuestos.
    if not Supuestos['Normalidad']['Cumple']:
        if JB_Pval < 0.001:
            Limitaciones.append("Violación severa de normalidad - considerar transformaciones de variables")
        else:
            Limitaciones.append("Violación leve de normalidad - los errores estándar robustos mitigan esto")
    
    if not Supuestos['Homocedasticidad']['Cumple']:
        Recomendaciones.append("Heterocedasticidad detectada - errores estándar robustos (HC3) apropiados")
    
    if not Supuestos['Independencia']['Cumple']:
        Limitaciones.append("Posible autocorrelación - considerar datos temporales o agrupados")
    
    if not Supuestos['Multicolinealidad']['Cumple']:
        Limitaciones.append(f"Multicolinealidad residual (VIF máximo = {Max_VIF:.2f})")
    
    # Recomendaciones basadas en tamaño de muestra.
    if N_Obs < 100:
        Limitaciones.append("Muestra pequeña - resultados menos generalizables")
    
    # Recomendaciones basadas en outliers.
    if Porcentaje_Outliers > 10:
        Recomendaciones.append("Investigar outliers - podrían indicar errores o subpoblaciones especiales")
    
    # Recomendaciones basadas en R².
    if R2_Ajustado < 0.30:
        Limitaciones.append("Bajo poder explicativo - considerar variables adicionales")
        Recomendaciones.append("Buscar predictores adicionales o interacciones entre variables")
    
    # Mostrar recomendaciones.
    if Recomendaciones:
        print("RECOMENDACIONES PRINCIPALES:")
        for i, Rec in enumerate(Recomendaciones, 1):
            print(f"  {i}. {Rec}")
    
    if Limitaciones:
        print(f"\nLIMITACIONES IDENTIFICADAS:")
        for i, Lim in enumerate(Limitaciones, 1):
            print(f"  {i}. {Lim}")
    
    # SECCIÓN 8: PUNTUACIÓN DE CONFIANZA.
    print(f"\n⭐ 8. PUNTUACIÓN DE CONFIANZA DEL MODELO")
    print("-"*50)
    
    # Calcular puntuación compuesta (0-100).
    Puntuacion = 0
    
    # Validez básica (40 puntos máximo).
    Puntuacion += (Criterios_Cumplidos / Total_Criterios) * 40
    
    # Supuestos (30 puntos máximo).
    Puntuacion += (Supuestos_Cumplidos / Total_Supuestos) * 30
    
    # Calidad de ajuste (20 puntos máximo).
    if R2_Ajustado >= 0.70:
        Puntuacion += 20
    elif R2_Ajustado >= 0.50:
        Puntuacion += 15
    elif R2_Ajustado >= 0.30:
        Puntuacion += 10
    elif R2_Ajustado >= 0.10:
        Puntuacion += 5
    
    # Outliers (10 puntos máximo).
    if Porcentaje_Outliers <= 5:
        Puntuacion += 10
    elif Porcentaje_Outliers <= 10:
        Puntuacion += 5
    
    # Determinar nivel de confianza.
    if Puntuacion >= 80:
        Nivel_Confianza = "🟢 ALTA CONFIANZA"
    elif Puntuacion >= 60:
        Nivel_Confianza = "🟡 CONFIANZA MODERADA"
    elif Puntuacion >= 40:
        Nivel_Confianza = "🟠 CONFIANZA BAJA"
    else:
        Nivel_Confianza = "🔴 CONFIANZA MUY BAJA"
    
    print(f"Puntuación total: {Puntuacion:.0f}/100")
    print(f"Nivel de confianza: {Nivel_Confianza}")
    
    # CONCLUSIÓN FINAL.
    print(f"\n🎯 CONCLUSIÓN FINAL")
    print("="*50)
    
    if Puntuacion >= 70:
        Conclusion = "El modelo es robusto y confiable para extraer conclusiones"
    elif Puntuacion >= 50:
        Conclusion = "El modelo es utilizable pero requiere interpretación cuidadosa"
    else:
        Conclusion = "El modelo necesita mejoras significativas antes de usarse"
    
    print(f"{Conclusion}")
    print(f"Variable dependiente '{Resultados_Modelo['Variable_Dependiente']}' ")
    print(f"puede ser explicada por {len(Variables_Sig)} predictores significativos")
    print(f"con R² ajustado de {R2_Ajustado:.3f} ({R2_Ajustado*100:.1f}% de varianza explicada)")
    
    # Retornar evaluación estructurada.
    Evaluacion_Estructurada = {
        'Validez_General': Validez_General.split(' ', 1)[1] if ' ' in Validez_General else Validez_General,
        'Calidad_Ajuste': Calidad_Ajuste.split(' ', 1)[1] if ' ' in Calidad_Ajuste else Calidad_Ajuste,
        'Cumplimiento_Supuestos': {nombre: datos['Cumple'] for nombre, datos in Supuestos.items()},
        'Interpretaciones_Coeficientes': Interpretaciones_Variables if len(Variables_Sig) > 0 else {},
        'Recomendaciones_Principales': Recomendaciones,
        'Limitaciones_Identificadas': Limitaciones,
        'Puntuacion_Confianza': Puntuacion,
        'Nivel_Confianza': Nivel_Confianza.split(' ', 1)[1] if ' ' in Nivel_Confianza else Nivel_Confianza
    }
    
    return Evaluacion_Estructurada